In [9]:
# 크롤링 코드 적용

import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time

def get_distributor(movie_title, headers):
    distributor = None
    try:
        query = quote_plus(f"영화 {movie_title} 정보")
        url = f"https://search.naver.com/search.naver?where=nexearch&query={query}"
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")
            info_items = soup.select("div.cm_content_area div.detail_info dl dt")
            for dt in info_items:
                if "배급" in dt.get_text(strip=True):
                    dd = dt.find_next_sibling("dd")
                    if dd:
                        distributor = dd.get_text(strip=True)
                        break
    except Exception as e:
        print(f"[{movie_title}] 배급사 오류: {e}")
    return distributor

def get_director(movie_title, headers):
    director = None

    # 1차 시도
    try:
        query_director = quote_plus(f"영화 {movie_title} 감독")
        url_director = f"https://search.naver.com/search.naver?where=nexearch&query={query_director}"
        res_director = requests.get(url_director, headers=headers)
        soup_director = BeautifulSoup(res_director.text, "html.parser")
        director_tag = soup_director.select_one("div.title_box strong.name a._text")
        if director_tag:
            director = director_tag.get_text(strip=True)
    except Exception as e:
        print(f"[{movie_title}] 감독 1차 오류: {e}")

    # 2차 시도
    if not director:
        try:
            query_cast = quote_plus(f"영화 {movie_title} 출연진")
            url_cast = f"https://search.naver.com/search.naver?where=nexearch&query={query_cast}"
            res_cast = requests.get(url_cast, headers=headers)
            soup_cast = BeautifulSoup(res_cast.text, "html.parser")
            director_tag = soup_cast.select_one("div.title_box > strong.name > span._text")
            if director_tag:
                director = director_tag.get_text(strip=True)
        except Exception as e:
            print(f"[{movie_title}] 감독 2차 오류: {e}")

    return director

# 📌 CSV 파일 불러오기
df = pd.read_csv("mergedRaw.csv")

# 📌 감독 또는 배급사가 비어있는 영화 중복 제거
needs_fill = df[df["감독"].isna() | df["배급사"].isna()]
unique_titles = needs_fill["영화명"].drop_duplicates().tolist()

# 📌 영화명 → {감독, 배급사} 결과 저장
info_dict = {}
headers = {"User-Agent": "Mozilla/5.0"}

for i, title in enumerate(unique_titles):
    existing = needs_fill[needs_fill["영화명"] == title].iloc[0]
    need_director = pd.isna(existing["감독"])
    need_distributor = pd.isna(existing["배급사"])

    director = None
    distributor = None

    if need_director:
        director = get_director(title, headers)
        time.sleep(3)

    if need_distributor:
        distributor = get_distributor(title, headers)
        time.sleep(3)

    info_dict[title] = {
        "감독": director if need_director else existing["감독"],
        "배급사": distributor if need_distributor else existing["배급사"]
    }

    print(f"[{i+1}/{len(unique_titles)}] {title} → 감독: {info_dict[title]['감독']}, 배급사: {info_dict[title]['배급사']}")
    time.sleep(3)

# 📌 원본 DataFrame의 결측치 채우기
def fill_missing(row):
    title = row["영화명"]
    if pd.isna(row["감독"]) and info_dict.get(title, {}).get("감독"):
        row["감독"] = info_dict[title]["감독"]
    if pd.isna(row["배급사"]) and info_dict.get(title, {}).get("배급사"):
        row["배급사"] = info_dict[title]["배급사"]
    return row

df_filled = df.apply(fill_missing, axis=1)

# 📌 새 파일로 저장
df_filled.to_csv("mergedRaw_filled.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: mergedRaw_filled.csv")


[1/1087] 캐벌케이드 → 감독: 요한 루프, 배급사: None
[2/1087] 유미의 세포들 더 무비 → 감독: 김다희, 배급사: 씨제이 씨지브이(CJ CGV)(주),롯데컬처웍스(주)롯데시네마
[3/1087] 아이다호 → 감독: 구스 반 산트, 배급사: None
[4/1087] 그 여름날의 누이 → 감독: 오시마 나기사, 배급사: None
[5/1087] 가라, 가라 두 번째 처녀 → 감독: 와카마츠 코지, 배급사: None
[6/1087] 청춘 스케치 → 감독: 벤 스틸러, 배급사: None
[7/1087] 모토신카카란누 → 감독: None, 배급사: None
[8/1087] 오키나와 드림쇼 → 감독: None, 배급사: None
[9/1087] 토스카 → 감독: 후고 데 아나, 배급사: None
[10/1087] 연애 지상주의 구역 Part 2 → 감독: None, 배급사: None
[11/1087] 연애 지상주의 구역 Part 1 → 감독: None, 배급사: None
[12/1087] 내가 마지막 본 파리 → 감독: 리차드 브룩스, 배급사: None
[13/1087] 적군-PFLP:세계전쟁선언 → 감독: 와카마츠 코지,아다치 마사오, 배급사: None
[14/1087] 카르멘 → 감독: 벤자민 마일피드, 배급사: 콘텐숍
[15/1087] 도쿄전쟁전후비화 → 감독: 오시마 나기사, 배급사: None
[16/1087] 초국지소지천황 → 감독: 하라 마사토, 배급사: None
[17/1087] 나비부인 → 감독: 안소니 밍겔라, 배급사: (주)위즈온센
[18/1087] 더 콕핏 → 감독: 미야케 쇼, 배급사: None
[19/1087] 성의 도적:섹스 잭 → 감독: 와카마츠 코지, 배급사: None
[20/1087] 즐거운 영혼 → 감독: 데이빗 린, 배급사: None
[21/1087] 피가로의 결혼 → 감독: 마틴 쿠세이, 배급사: None
[22/1087] 워터 프론트 → 감독: 존 소그렌, 배급사: None
[23/1087] 요조숙녀 → 감독: 프

[176/1087] 야마부키 → 감독: 야마사키 쥬이치로, 배급사: None
[177/1087] 스탠 바이 미 → 감독: 롭 라이너, 배급사: None
[178/1087] 붉은 혜성 클럽 → 감독: None, 배급사: None
[179/1087] 빅샤크4: 바다공룡 대모험 → 감독: 장초, 배급사: 와이드 릴리즈(주)
[180/1087] 살인자들 → 감독: 로버트 시오드막, 배급사: ㈜미디어 마그나
[181/1087] 레슨 → 감독: 김경래, 배급사: None
[182/1087] 2024 박은빈 팬 콘서트 → 감독: None, 배급사: 씨제이 씨지브이(CJ CGV)(주)
[183/1087] 이솝이야기 → 감독: 서윤미, 배급사: 주식회사 푸른잎커뮤니케이션
[184/1087] 빅샤크5: 80일간의 해저일주 → 감독: 첸융강, 배급사: (주)팝엔터테인먼트
[185/1087] 사막의 두 남자 → 감독: 파우지 벤사이디, 배급사: None
[186/1087] 인샬라 어 보이 → 감독: 암자드 알 라시드, 배급사: None
[187/1087] 오만 단편선 → 감독: None, 배급사: None
[188/1087] 열차 안의 낯선 자들, 의혹의 전망차 → 감독: 알프레드 히치콕, 배급사: None
[189/1087] 황금팔을 가진 사나이 → 감독: 오토 프레밍거, 배급사: None
[190/1087] 핫잔: 낙타 기수 마타르 → 감독: 아부 바크르 쇼키, 배급사: None
[191/1087] 내 곁에 있어줘 → 감독: 황웬잉, 배급사: None
[192/1087] 나의 선생님 → 감독: 파라 나불시, 배급사: None
[193/1087] 굿바이 줄리아 → 감독: 모하메드 코르도파니, 배급사: None
[194/1087] 마지막 여왕 → 감독: 데미앙 은누리, 배급사: None
[195/1087] 썸머프라이드시네마 2024 단편1 → 감독: None, 배급사: None
[196/1087] 썸머프라이드시네마 2024 김예창 배우 특별전 → 감독: None, 배급사: None

[347/1087] 이구알라다, 쟁취하는 자 → 감독: 쥬앙 메지아 보테로, 배급사: None
[348/1087] 마리아, 나로 살기 → 감독: 제시카 팔루드, 배급사: None
[349/1087] 오색의 린 → 감독: 이원우, 배급사: None
[350/1087] 우리의 땅, 우리의 자유 → 감독: 미나 난지, 배급사: None
[351/1087] 나쁜 날씨가 좋아서 → 감독: 야나 레카르스카, 배급사: None
[352/1087] 어떤 프랑스 청년 → 감독: 제레미 바타글리아, 배급사: None
[353/1087] 비바 바르다! → 감독: 피에르 앙리 지베르, 배급사: None
[354/1087] [SIWFF] 경계에서, 끈질기게 단편선 → 감독: None, 배급사: None
[355/1087] [SIWFF] 아시아단편 단편선 2 → 감독: None, 배급사: None
[356/1087] [SIWFF] 애니메이티드, 몸-세계-존재 단편선 3 → 감독: None, 배급사: None
[357/1087] [SIWFF] 아시아단편 단편선 3 → 감독: None, 배급사: None
[358/1087] 아리랑 랩소디 → 감독: 김성웅, 배급사: None
[359/1087] 언제나 그 자리에 + 완벽한 하루 → 감독: None, 배급사: None
[360/1087] 귀 기울여 들으면 → 감독: 지아인 리우, 배급사: None
[361/1087] 키스펩틴 연대기 → 감독: 오카다 시가, 배급사: None
[362/1087] [SIWFF] 아시아단편 단편선 4 → 감독: None, 배급사: None
[363/1087] 축구 소녀 모나 → 감독: 솔린 유세프, 배급사: None
[364/1087] [SIWFF] 애니메이티드, 몸-세계-존재 단편선 4 → 감독: None, 배급사: None
[365/1087] 봄의 피부 → 감독: 니퍼 우리베 알자테, 배급사: None
[366/1087] 아일랜드 인 비트윈 + 석양의 빛: 디아스포라 이야기 → 감독: None, 배급사: No

[510/1087] 암리타의 향연 → 감독: 사카모토 사쿠, 배급사: None
[511/1087] 빌어먹을 휘게 → 감독: 안나 카제약, 배급사: None
[512/1087] 제16회 서울영등포국제초단편영화제 초·단편 경쟁 5 → 감독: None, 배급사: None
[513/1087] 제16회 서울영등포국제초단편영화제 초·단편 경쟁 8 → 감독: None, 배급사: None
[514/1087] 푸른 장벽 → 감독: 아그네츠카 홀란드, 배급사: None
[515/1087] 제20회 서울인디애니페스트2024 아시아로2 → 감독: None, 배급사: None
[516/1087] 제20회 서울인디애니페스트2024 사리나 니헤이: 이상한 나라의 기괴한 이야기들 → 감독: 사리나 니헤이, 배급사: None
[517/1087] 모뉴먼트 → 감독: 윌리엄 켄트리지, 배급사: None
[518/1087] 구스타보 두다멜 & 아스믹 그리고리안 → 감독: 구스타보 두다멜, 배급사: None
[519/1087] 제20회 서울인디애니페스트2024 국내초청1 → 감독: None, 배급사: None
[520/1087] 제20회 서울인디애니페스트2024 독립보행3 → 감독: None, 배급사: None
[521/1087] 제20회 서울인디애니페스트2024 파노라마2 : 물 만난 단편선 → 감독: None, 배급사: None
[522/1087] 제16회 서울영등포국제초단편영화제 초·단편 경쟁 3 → 감독: None, 배급사: None
[523/1087] 예술대학 1994 → 감독: 리우 지앤, 배급사: None
[524/1087] 제16회 서울영등포국제초단편영화제 초·단편 경쟁 2 → 감독: None, 배급사: None
[525/1087] 제20회 서울인디애니페스트2024 새벽비행2 → 감독: None, 배급사: None
[526/1087] 제20회 서울인디애니페스트2024 아시아로4 → 감독: None, 배급사: None
[527/1087] 제16회 서울영등포국제초단편영화제 초·단편 경쟁

[679/1087] 란위 → 감독: 관금붕, 배급사: None
[680/1087] 아라가네 → 감독: 오다 카오리, 배급사: None
[681/1087] SIPFF2024 국내단편 경쟁 1 → 감독: None, 배급사: None
[682/1087] 솔로 → 감독: 소피 듀피스, 배급사: ㈜에이블엔터테인먼트
[683/1087] 가마 → 감독: 오다 카오리, 배급사: None
[684/1087] SIPFF2024 국내단편 비경쟁 1 → 감독: None, 배급사: None
[685/1087] 솔직하지 못해서 → 감독: 이진용, 배급사: None
[686/1087] 세바스티앙 → 감독: None, 배급사: None
[687/1087] 세노테 → 감독: 오다 카오리, 배급사: None
[688/1087] 다정함을 향해 → 감독: 오다 카오리, 배급사: None
[689/1087] 그러므로 노이즈가 말한다+호응+플래시 → 감독: 오다 카오리, 배급사: None
[690/1087] 스타워즈 키드 → 감독: 마티외 푸르니에, 배급사: None
[691/1087] 소풍 → 감독: 김용균, 배급사: 롯데엔터테인먼트
[692/1087] 컷 슬리브 보이스 → 감독: 레이 영, 배급사: None
[693/1087] SBFF2024 단편경쟁 8 → 감독: None, 배급사: None
[694/1087] SBFF2024 단편경쟁 9 → 감독: None, 배급사: None
[695/1087] SBFF2024 → 감독: None, 배급사: None
[696/1087] SBFF2024 성북시퀀스 옴니버스 → 감독: None, 배급사: None
[697/1087] 프론트 커버 → 감독: 레이 영, 배급사: None
[698/1087] 아저씨 x 아저씨 → 감독: 레이 영, 배급사: None
[699/1087] 저승과 진자 → 감독: 로저 코만, 배급사: None
[700/1087] 우중산책 → 감독: 임순례, 배급사: None
[701/1087] SBFF2024 성북시퀀스 2 → 감독: Non

[847/1087] 길다 → 감독: 찰스 비더, 배급사: None
[848/1087] 브리드 → 감독: 스테폰 브리스톨, 배급사: 주식회사 풍경소리
[849/1087] 센트로 히스토리코 → 감독: 마누엘 데 올리베이라,페드로 코스타,빅토르 에리세,아키 카우리스마키, 배급사: None
[850/1087] 추억의 장미 → 감독: 다니엘 만, 배급사: None
[851/1087] 햇빛 속의 모과나무 → 감독: 빅토르 에리세, 배급사: None
[852/1087] 제16회 스토리업 쇼츠: 칸영화제 특별편 → 감독: None, 배급사: None
[853/1087] 이스터 퍼레이드 → 감독: 찰스 월터스, 배급사: None
[854/1087] 편지 → 감독: 윌리엄 와일러, 배급사: 아트시네마
[855/1087] 헤어졌을때와 만났을때 → 감독: 데이빗 스위프트, 배급사: None
[856/1087] 호두까기 인형 → 감독: 안드레이 콘찰로프스키, 배급사: (주)위즈온센
[857/1087] 12월 수요단편극장 FILM IT! → 감독: None, 배급사: None
[858/1087] 인디언 파이터 → 감독: 안드레 드 토스, 배급사: None
[859/1087] 제3회 서울문화고 문화영상제 - 1학년 작품 → 감독: None, 배급사: None
[860/1087] 제3회 서울문화고 문화영상제 - 3학년 작품 → 감독: None, 배급사: None
[861/1087] 제3회 서울문화고 문화영상제 - 2학년 작품 → 감독: None, 배급사: None
[862/1087] 순간에서 영원으로 → 감독: None, 배급사: None
[863/1087] 미생: 더할 나위 없었다 → 감독: 김원석, 배급사: 씨지브이 아이스콘
[864/1087] 우주에서, 메리 크리스마스 → 감독: 낮잠, 배급사: 씨제이 씨지브이(CJ CGV)(주)
[865/1087] 비숍스 와이프 → 감독: 헨리 코스터, 배급사: None
[866/1087] 챌린지스 → 감독: 빅토르 에리세, 배급사: None
[8

[1018/1087] 택시 드라이버 → 감독: 마틴 스코세이지, 배급사: None
[1019/1087] 류이치 사카모토: 플레잉 디 오케스트라 2014 → 감독: 나나미 오노, 배급사: 리벳
[1020/1087] 인투 더 원더우즈 → 감독: 뱅상 파로노드, 배급사: None
[1021/1087] 백설공주와 일곱난쟁이 → 감독: 에리카 듀크, 배급사: (주)박수엔터테인먼트
[1022/1087] 서울독립영화제 순회상영회 인디피크닉2025 in 서울 단편 4 → 감독: None, 배급사: None
[1023/1087] 자니 기타 → 감독: 니콜라스 레이, 배급사: None
[1024/1087] 서울독립영화제 순회상영회 인디피크닉2025 in 서울 단편 2 → 감독: None, 배급사: None
[1025/1087] 서울독립영화제 순회상영회 인디피크닉2025 in 서울 단편 1 → 감독: None, 배급사: None
[1026/1087] 서울독립영화제 순회상영회 인디피크닉2025 in 서울 단편 6 → 감독: None, 배급사: None
[1027/1087] 서울독립영화제 순회상영회 인디피크닉2025 in 서울 단편 5 → 감독: None, 배급사: None
[1028/1087] 서울독립영화제 순회상영회 인디피크닉2025 in 서울 단편 3 → 감독: None, 배급사: None
[1029/1087] 용장 로모로 → 감독: 세르지오 코르부치, 배급사: None
[1030/1087] 길은 멀어도 마음만은 → 감독: 루이스 루시아, 배급사: None
[1031/1087] 뱀파이어 → 감독: 칼 테오도르 드레이어, 배급사: None
[1032/1087] 당신과 나를 잇는 법 → 감독: 김윤겸,여인서,임수빈, 배급사: None
[1033/1087] 2025년 4월 수요단편극장 - 4월은 너의 거짓말 → 감독: None, 배급사: None
[1034/1087] 분노의 날 → 감독: 칼 테오도르 드레이어, 배급사: None
[1035/1087] 4월에는 사랑을 말할 것 (섹션2

In [5]:
# 크롤링 코드

import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time

def get_movie_info_from_naver(movie_title):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    ### 🎬 1. 배급사 추출
    distributor = None
    query_info = quote_plus(f"영화 {movie_title} 정보")
    url_info = f"https://search.naver.com/search.naver?where=nexearch&query={query_info}"
    try:
        res_info = requests.get(url_info, headers=headers)
        soup_info = BeautifulSoup(res_info.text, "html.parser")
        info_items = soup_info.select("div.cm_content_area div.detail_info dl dt")
        for dt in info_items:
            if "배급" in dt.get_text(strip=True):
                dd = dt.find_next_sibling("dd")
                if dd:
                    distributor = dd.get_text(strip=True)
                    break
    except Exception as e:
        print(f"[{movie_title}] 배급사 오류: {e}")

    ### 🎬 2. 감독 추출
    director = None

    # ▶ 1차 시도: "영화 {movie_title} 감독" 검색
    try:
        query_director = quote_plus(f"영화 {movie_title} 감독")
        url_director = f"https://search.naver.com/search.naver?where=nexearch&query={query_director}"
        res_director = requests.get(url_director, headers=headers)
        soup_director = BeautifulSoup(res_director.text, "html.parser")

        director_tag = soup_director.select_one("div.title_box strong.name a._text")
        if director_tag:
            director = director_tag.get_text(strip=True)
    except Exception as e:
        print(f"[{movie_title}] 감독 1차 오류: {e}")

    # ▶ 2차 시도: "f'{movie_title} 영화 출연진'" 검색
    if not director:
        try:
            query_cast = quote_plus(f"{movie_title} 영화 출연진")
            url_cast = f"https://search.naver.com/search.naver?where=nexearch&query={query_cast}"
            res_cast = requests.get(url_cast, headers=headers)
            soup_cast = BeautifulSoup(res_cast.text, "html.parser")

            director_tag = soup_cast.select_one("div.title_box > strong.name > span._text")
            if director_tag:
                director = director_tag.get_text(strip=True)
        except Exception as e:
            print(f"[{movie_title}] 감독 2차 오류: {e}")

    ### ✅ 결과 출력
    print(f"🎞️ [{movie_title}]")
    print(f"  🎬 감독: {director if director else '정보 없음'}")
    print(f"  🏢 배급사: {distributor if distributor else '정보 없음'}")
    print("-" * 40)

    return {
        "title": movie_title,
        "director": director,
        "distributor": distributor
    }

# ✅ 테스트 실행
movie_titles = [
    "올파의 딸들",
    "성 베드로 대성당과 로마의 교황청 대성당들",
    "아도 스페셜 라이브 신조우 인 시네마",
    "은혼 더 파이널"
]

for title in movie_titles:
    get_movie_info_from_naver(title)
    time.sleep(3)


🎞️ [올파의 딸들]
  🎬 감독: 카우타르 벤 하니야
  🏢 배급사: 필름다빈
----------------------------------------
🎞️ [성 베드로 대성당과 로마의 교황청 대성당들]
  🎬 감독: 루카 비오토
  🏢 배급사: 일미디어
----------------------------------------
🎞️ [아도 스페셜 라이브 신조우 인 시네마]
  🎬 감독: 히로세 토시히토
  🏢 배급사: 와이드 릴리즈㈜
----------------------------------------
🎞️ [은혼 더 파이널]
  🎬 감독: 미야와키 치즈루
  🏢 배급사: 워터홀컴퍼니㈜
----------------------------------------
